# Trial by trial variability analysis

**11th March 2024 (edited the 6th of May 2024)**

*Théo Gauvrit & Célien Vandromme*

---

## Modules and data import

In [ ]:
import numpy as np
import pandas as pd
import percephone.core.recording as pc
import os
import percephone.plts.behavior as pbh
import matplotlib
import percephone.plts.stats as ppt
import matplotlib.pyplot as plt
import percephone.analysis.mlr_models as mlr_m
from multiprocessing import Pool, cpu_count, pool
from sklearn.metrics.pairwise import cosine_similarity
plt.rcParams['font.size'] = 10
plt.rcParams['axes.linewidth'] = 2
plt.switch_backend("Qt5Agg")
matplotlib.use("Qt5Agg")
%matplotlib inline

In [ ]:
user = "Célien"

if user == "Célien":
    directory = "C:/Users/cvandromme/Desktop/Data/"
    roi_path = "C:/Users/cvandromme/Desktop/FmKO_ROIs&inhibitory.xlsx"
elif user == "Théo":
    directory = "/datas/Théo/Projects/Percephone/data/Amplitude_Detection/loop_format_tau_02/"
    roi_path = directory + "/FmKO_ROIs&inhibitory.xlsx"
    
roi_info = pd.read_excel(roi_path)
files = os.listdir(directory)
files_ = [file for file in files if file.endswith("synchro")]

def opening_rec(fil, i):
    rec = pc.RecordingAmplDet(directory + fil + "/", 0, roi_path)
    rec.peak_delay_amp()
    return rec

workers = cpu_count()
if user == "Célien":
    pool = pool.ThreadPool(processes=workers)
elif user == "Théo":
    pool = Pool(processes=workers)
async_results = [pool.apply_async(opening_rec, args=(file, i)) for i, file in enumerate(files_)]
recs = {ar.get().filename: ar.get() for ar in async_results}

## Functions used

Function that will generate the cosine similarity matrix

In [ ]:
def cosine_matrix(ax, rec, amplitude, resp_mat):
    print("Cosine similarity computation")

    sim_mat = cosine_similarity(resp_mat)
    ax.imshow(sim_mat, cmap="seismic", vmin=-1, vmax=+1, interpolation="none")
    ax.set_xlabel("Trial i")
    ax.set_ylabel("Trial j")
    ax.set_title(str(rec.filename) + " " + rec.genotype + " " + str(amplitude))
    ax.grid(False)

Functions used to estimate the zscore during stimulation time

In [ ]:
def neuron_mean_std_corr(array, estimator):
    if estimator == "Mean":
        return np.mean(array, axis=0)
    if estimator == "Std":
        return np.std(array, axis=0)
    

def get_zscore(rec, exc_neurons=True, inh_neurons=False):
        if exc_neurons and inh_neurons:
            zscore = np.row_stack((rec.zscore_exc, rec.zscore_inh)).T
        elif exc_neurons:
            zscore = rec.zscore_exc.T
        elif inh_neurons:
            zscore = rec.zscore_inh.T
        return zscore

def get_zscore_estimator(rec, estimator, exc_neurons=True, inh_neurons=True):
    zscore = get_zscore(rec, exc_neurons=exc_neurons, inh_neurons=inh_neurons)
    first = True
    for i in range(rec.stim_time.shape[0]):
        start = rec.stim_time[i]
        end = rec.stim_time[i] + int(rec.stim_durations[i])
        if first:
            X = neuron_mean_std_corr(zscore[start : end], estimator)
            first = False
        else:
            new_row = neuron_mean_std_corr(zscore[start : end], estimator)
            X = np.row_stack((X, new_row))
    return X

The function used to plot the cosine similarity

In [ ]:
def plot_cosine_similarity(recs, parameter, estimator=None, exc_neurons=True, inh_neurons=False, detected_trials=True, undetected_trials=True, stim_ampl="all"):
    y, i = 0, 0
    fig, ax = plt.subplots(4, 8, figsize=(35, 20))
    
    for rec in recs.values():
        # Filtering stimulation amplitudes
        stim_filter = rec.stim_ampl_filter(stim_ampl)
        
        # Retrieving data for specified parameter and neuron type
        if parameter == "zscore":
            para_mat = get_zscore_estimator(rec, estimator, exc_neurons=exc_neurons, inh_neurons=inh_neurons).T
            neuron_type = "EXC & INH" if (exc_neurons and inh_neurons) else ("EXC" if exc_neurons else ("INH" if inh_neurons else "NONE"))
        else:
            # Filtering neuron type
            if exc_neurons and inh_neurons: 
                para_mat = np.row_stack((rec.matrices["EXC"][parameter], rec.matrices["INH"][parameter]))
                neuron_type = "EXC & INH"
            elif exc_neurons:
                para_mat = rec.matrices["EXC"][parameter]
                neuron_type = "EXC"
            elif inh_neurons:
                para_mat = rec.matrices["INH"][parameter]
                neuron_type = "INH"
        
        # Filtering detected trials and amplitudes
        if detected_trials and undetected_trials:
            filt_para_mat = para_mat.T[stim_filter]
            trials_type = "all"
        elif detected_trials:
            filt_para_mat = para_mat.T[np.logical_and(rec.detected_stim, stim_filter)]
            trials_type = "detected"
        elif undetected_trials:
            filt_para_mat = para_mat.T[np.logical_and(np.invert(rec.detected_stim), stim_filter)]
            trials_type = "undetected"
        
        if rec.genotype == "WT":
            pbh.psycho_like_plot(rec, roi_info, ax[0, i])
            cosine_matrix(ax[1, i], rec, rec.threshold, filt_para_mat)
            i = i + 1
        else:
            pbh.psycho_like_plot(rec, roi_info, ax[2, y])
            cosine_matrix(ax[3, y], rec, rec.threshold, filt_para_mat)
            y = y + 1
    
    # Hiding the empty graphs
    ax[2, 6].set_axis_off()
    ax[2, 7].set_axis_off()
    ax[3, 6].set_axis_off()
    ax[3, 7].set_axis_off()
    if parameter == "zscore":
        fig.suptitle(f"Cosine similarity based on {estimator} {parameter} for {trials_type} trials in {neuron_type} neurons. Amplitude(s): {stim_ampl}", fontsize=26)
    else:
        fig.suptitle(f"Cosine similarity based on {parameter} for {trials_type} trials in {neuron_type} neurons. Amplitude(s): {stim_ampl}", fontsize=26)

# Cosine similarity based on responsivity

---

## All amplitudes

### All trials

In [ ]:
plot_cosine_similarity(recs, "Responsivity", estimator=None, exc_neurons=True, inh_neurons=False, detected_trials=True, undetected_trials=True, stim_ampl="all")

### Detected trials

In [ ]:
plot_cosine_similarity(recs, "Responsivity", estimator=None, exc_neurons=True, inh_neurons=False, detected_trials=True, undetected_trials=False, stim_ampl="all")

### Undetected trials

In [ ]:
plot_cosine_similarity(recs, "Responsivity", estimator=None, exc_neurons=True, inh_neurons=False, detected_trials=False, undetected_trials=True, stim_ampl="all")

## Threshold amplitude

### All trials

In [ ]:
plot_cosine_similarity(recs, "Responsivity", estimator=None, exc_neurons=True, inh_neurons=False, detected_trials=True, undetected_trials=True, stim_ampl="threshold")

### Detected trials

In [ ]:
plot_cosine_similarity(recs, "Responsivity", estimator=None, exc_neurons=True, inh_neurons=False, detected_trials=True, undetected_trials=False, stim_ampl="threshold")

### Undetected trials

In [ ]:
plot_cosine_similarity(recs, "Responsivity", estimator=None, exc_neurons=True, inh_neurons=False, detected_trials=False, undetected_trials=True, stim_ampl="threshold")

# Cosine similarity based on mean zscore

---

## All amplitudes

### All trials

In [ ]:
plot_cosine_similarity(recs, "zscore", estimator="Mean", exc_neurons=True, inh_neurons=False, detected_trials=True, undetected_trials=True, stim_ampl="all")

### Detected trials

In [ ]:
plot_cosine_similarity(recs, "zscore", estimator="Mean", exc_neurons=True, inh_neurons=False, detected_trials=True, undetected_trials=False, stim_ampl="all")

### Undetected trials

In [ ]:
plot_cosine_similarity(recs, "zscore", estimator="Mean", exc_neurons=True, inh_neurons=False, detected_trials=False, undetected_trials=True, stim_ampl="all")

## Threshold amplitude

### All trials

In [ ]:
plot_cosine_similarity(recs, "zscore", estimator="Mean", exc_neurons=True, inh_neurons=False, detected_trials=True, undetected_trials=True, stim_ampl="threshold")

### Detected trials

In [ ]:
plot_cosine_similarity(recs, "zscore", estimator="Mean", exc_neurons=True, inh_neurons=False, detected_trials=True, undetected_trials=False, stim_ampl="threshold")

### Undetected trials

In [ ]:
plot_cosine_similarity(recs, "zscore", estimator="Mean", exc_neurons=True, inh_neurons=False, detected_trials=False, undetected_trials=True, stim_ampl="threshold")

# Cosine similarity based on std of zscore

---

## All amplitudes

### All trials

In [ ]:
plot_cosine_similarity(recs, "zscore", estimator="Std", exc_neurons=True, inh_neurons=False, detected_trials=True, undetected_trials=True, stim_ampl="all")